In [ ]:
#!pip install transformers datasets peft trl bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 16.4 MB/s eta 0:00:00


# Loading the dataset and preprocessing:

In [ ]:
from datasets import load_dataset

ds = load_dataset("eswardivi/medical_qa")
ds = ds.remove_columns(["input"]).rename_column("instruction", "input")
train_test_split = ds["train"].train_test_split(test_size=307, train_size=6000, shuffle=True, seed=42)
new_ds = {"train": train_test_split["train"], "test": train_test_split["test"]}

# Verify the changes
print(new_ds)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

data_merged.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/6307 [00:00<?, ? examples/s]

{'train': Dataset({
    features: ['input', 'output'],
    num_rows: 6000
}), 'test': Dataset({
    features: ['input', 'output'],
    num_rows: 307
})}


In [ ]:
new_ds['train'][2]

{'input': "I'm aware that people are recommended to stay away from taking ibuprofen, cortisone, and other anti-inflammatory drugs. But does this also apply to supplements? For example, St. John's Wort, green tea, and turmeric are all supposed to have anti-inflammatory properties. Would it be better to avoid or limit their consumption at this time?",
 'output': "Most health care groups have not made any recommendations regarding ibuprofen or other NSAIDs. We simply don't have the information to be able to recommend one way or another. If you have been taking these supplements, it should be fine to continue taking them."}

In [ ]:
#!pip install bitsandbytes==0.47.0 --force-reinstall

# Loading the Qwen 2.5-0.5B model and Initializing the quantization for QLoRA:

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
import torch

model_name = "Qwen/Qwen2.5-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quant_config,
    trust_remote_code=True
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [ ]:
#pip install transformers datasets peft trl bitsandbytes accelerate

# Finetuning the Model:

In [ ]:
from transformers import TrainingArguments
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

In [ ]:
def prompt_instruction_format(sample):
    return f"""### Instruction:
Provide a concise and accurate medical answer in one sentence based on the input below. If the information is unknown, respond with 'I don’t know.'

### Input:
{sample['input']}

### Response:
{sample['output']} <|END_OF_TEXT|>"""

In [ ]:
peft_config = LoraConfig(
    r=16,  # Keep 16, or try 8 for less capacity, 32 for more
    lora_alpha=32,  # Adjust to 16 or 64 based on convergence
    lora_dropout=0.1,  # Increase to 0.1 for more regularization
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
)

In [ ]:
model = get_peft_model(model, peft_config)

In [ ]:
training_args = TrainingArguments(
    output_dir="./output",
    num_train_epochs=5,  # Increase to 5 for better convergence
    per_device_train_batch_size=2,  # Increase to fit T4 VRAM
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,  # Reduce to 2 for faster updates
    learning_rate=1e-4,  # Lower to stabilize training
    warmup_steps=50,  # Increase for smoother learning curve
    logging_steps=10,
    save_strategy="epoch",  # Save at each epoch
    eval_strategy="steps",  # Enable evaluation
    eval_steps=50,
    fp16=True,
    optim="adamw_8bit",
    report_to="none"
)

In [ ]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=new_ds["train"],
    eval_dataset=new_ds["test"],
    peft_config=peft_config,
    formatting_func=prompt_instruction_format,
    args=training_args,
)

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Applying formatting function to train dataset:   0%|          | 0/6000 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/6000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/6000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/6000 [00:00<?, ? examples/s]

Applying formatting function to eval dataset:   0%|          | 0/307 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/307 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/307 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/307 [00:00<?, ? examples/s]

In [ ]:
# Train and save
trainer.train()



The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
50,2.242700,2.156028,2.121865,30640.000000,0.590561
100,1.825000,1.907057,1.915966,59373.000000,0.622318
150,1.591700,1.865085,1.827957,90587.000000,0.627935
200,1.761600,1.842639,1.826931,123426.000000,0.630273
250,1.543600,1.826534,1.852806,153049.000000,0.631868
300,1.769200,1.815424,1.834019,182798.000000,0.634429
350,1.725600,1.806458,1.754212,212442.000000,0.636704
400,1.736400,1.790255,1.796339,241703.000000,0.637528
450,1.812600,1.783649,1.773525,272373.000000,0.636947
500,1.860800,1.777103,1.811481,298850.000000,0.640063


NameError: name 'drive' is not defined

In [ ]:
from google.colab import drive

# Mount Google Drive and save
drive.mount('/content/drive')
output_dir = "/content/drive/MyDrive/fine_tuned_qwen_medical_qa"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Mounted at /content/drive


('/content/drive/MyDrive/fine_tuned_qwen_medical_qa/tokenizer_config.json',
 '/content/drive/MyDrive/fine_tuned_qwen_medical_qa/special_tokens_map.json',
 '/content/drive/MyDrive/fine_tuned_qwen_medical_qa/chat_template.jinja',
 '/content/drive/MyDrive/fine_tuned_qwen_medical_qa/vocab.json',
 '/content/drive/MyDrive/fine_tuned_qwen_medical_qa/merges.txt',
 '/content/drive/MyDrive/fine_tuned_qwen_medical_qa/added_tokens.json',
 '/content/drive/MyDrive/fine_tuned_qwen_medical_qa/tokenizer.json')

In [ ]:
#!pip install huggingface_hub

In [ ]:

from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import os
from huggingface_hub import HfApi

local_dir = "./fine_tuned_qwen_medical_qa_updated"
os.makedirs(local_dir, exist_ok=True)

model.save_pretrained(local_dir)
tokenizer.save_pretrained(local_dir)

# 2. Create/ensure the Hugging Face repo exists
api = HfApi()
repo_id = "PavansaiGundaram/fine_tuned_qwen_medical_qa_updated_version"  # your HF username/repo
api.create_repo(repo_id=repo_id, repo_type="model", exist_ok=True)

# 3. Push the new local directory to HF Hub
api.upload_folder(
    folder_path=local_dir,
    repo_id=repo_id,
    repo_type="model",
    commit_message="Upload fine-tuned Qwen2.5-0.5B (updated version)"
)

print(f"✅ Model pushed to https://huggingface.co/{repo_id}")


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...a_updated/adapter_model.safetensors:   0%|          | 21.4kB / 8.68MB            

  ...n_medical_qa_updated/tokenizer.json: 100%|##########| 11.4MB / 11.4MB            

✅ Model pushed to https://huggingface.co/PavansaiGundaram/fine_tuned_qwen_medical_qa_updated_version
